The code for this segment has been influenced by Dataquest. Certain actions such as the scrapping of both away and home stats using the [0] and [1] index have been added.


In [ ]:
import pandas as pd

In [ ]:
#Shooting 
import requests
from bs4 import BeautifulSoup
import time


years = list(range(2022, 2016, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        poss_for = pd.read_html(data.text, match="Shooting")[0]
        poss_against = pd.read_html(data.text, match="Shooting")[1]
        poss_for.columns = poss_for.columns.droplevel()
        poss_against.columns = poss_against.columns.droplevel()
        try:
            team_data1 = matches.merge(poss_for[["Date","Sh", "SoT","Dist", "FK", "PK", "PKatt"]], on="Date")
            team_data_shooting = team_data1.merge(poss_against[["Date","Sh", "SoT","Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data_shooting[team_data_shooting["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data_shooting)
        time.sleep(15)

In [4]:
combined1_df = pd.concat(all_matches, ignore_index=True)

# Print the combined DataFrame
combined1_df

combined1_df.to_csv('combined_matches_shot.csv', index=False)

In [5]:
combined1_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist_x,FK_x,PK_x,PKatt_x,Sh_y,SoT_y,Dist_y,FK_y,PK_y,PKatt_y
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (4),1 (1),Manchester City,...,NaN,NaN,0,0,8.0,4.0,NaN,NaN,0,0
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,19.1,0.0,0,0,6.0,2.0,17.3,0.0,0,0
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,16.4,0.0,1,1,14.0,2.0,16.4,1.0,0,0
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,13.8,0.0,1,1,8.0,3.0,17.8,0.0,0,0
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,15.0,0.0,0,0,10.0,2.0,16.5,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5644,2019-04-13,12:30,Premier League,Matchweek 34,Sat,Away,L,0,4,Tottenham,...,18.9,1.0,0,0,22.0,5.0,16.1,1.0,0,0
5645,2019-04-20,15:00,Premier League,Matchweek 35,Sat,Home,L,1,2,Watford,...,18.1,1.0,0,0,11.0,6.0,18.1,1.0,0,0
5646,2019-04-26,20:00,Premier League,Matchweek 36,Fri,Away,L,0,5,Liverpool,...,21.6,0.0,0,0,21.0,7.0,16.6,0.0,0,0
5647,2019-05-05,14:00,Premier League,Matchweek 37,Sun,Home,D,1,1,Manchester Utd,...,19.0,1.0,0,0,23.0,7.0,19.0,2.0,0,0


In [ ]:
#Possession
import requests
from bs4 import BeautifulSoup
import time


years = list(range(2022, 2016, -1))

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/possession/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        poss_for = pd.read_html(data.text, match="Possession")[0]
        poss_against = pd.read_html(data.text, match="Possession")[1]
        poss_for.columns = poss_for.columns.droplevel()
        poss_against.columns = poss_against.columns.droplevel()
        try:
            team_data1 = matches.merge(poss_for[["Date","Poss","Succ%","CPA"]], on="Date")
            team_data = team_data1.merge(poss_against[["Date","Poss","Succ%","CPA"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(15)

In [ ]:
combined2_df = pd.concat(all_matches, ignore_index=True)

# Print the combined DataFrame
combined2_df

combined2_df.to_csv('combined_matches_posses.csv', index=False)

In [ ]:
#passing
import requests
from bs4 import BeautifulSoup
import time


years = list(range(2022, 2016, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/passing/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        poss_for = pd.read_html(data.text, match="Passing")[0]
        poss_against = pd.read_html(data.text, match="Passing")[1]
        poss_for.columns = poss_for.columns.droplevel()
        poss_against.columns = poss_against.columns.droplevel()
        try:
            team_data1 = matches.merge(poss_for[["Date","KP","1/3","PPA","Prog,CrsPA"]], on="Date")
            team_data = team_data1.merge(poss_against[["Date","KP","1/3","PPA","Prog,CrsPA"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(15)

In [ ]:
combined3_df = pd.concat(all_matches, ignore_index=True)

# Print the combined DataFrame
combined3_df

combined3_df.to_csv('combined_matches_passing.csv', index=False)

In [ ]:
#def
import requests
from bs4 import BeautifulSoup
import time


years = list(range(2022, 2016, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/defense/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        poss_for = pd.read_html(data.text, match="Defensive Actions")[0]
        poss_against = pd.read_html(data.text, match="Defensive Actions")[1]
        poss_for.columns = poss_for.columns.droplevel()
        poss_against.columns = poss_against.columns.droplevel()
        try:
            team_data1 = matches.merge(poss_for[["Date","Tkl","TklW","Def 3rd","Tkl%","Blocks","Sh","Int","Err"]], on="Date")
            team_data = team_data1.merge(poss_against[["Date","Tkl","TklW","Def 3rd","Tkl%","Blocks","Sh","Int","Err"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(15)

In [ ]:
combined4_df = pd.concat(all_matches, ignore_index=True)

# Print the combined DataFrame
combined4_df

combined4_df.to_csv('combined_matches_defend.csv', index=False)

In [ ]:
#SCA
import requests
from bs4 import BeautifulSoup
import time


years = list(range(2022, 2016, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/gca/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        poss_for = pd.read_html(data.text, match="Goal and Shot Creation")[0]
        poss_against = pd.read_html(data.text, match="Goal and Shot Creation")[1]
        poss_for.columns = poss_for.columns.droplevel()
        poss_against.columns = poss_against.columns.droplevel()
        try:
            team_data1 = matches.merge(poss_for[["Date","SCA", "PassLive" ,"TO"]], on="Date")
            team_data = team_data1.merge(poss_against[["Date","SCA", "PassLive" ,"TO"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(15)

In [ ]:
combined5_df = pd.concat(all_matches, ignore_index=True)
# Print the combined DataFrame
combined5_df
combined5_df.to_csv('combined_matches_sca.csv', index=False)